In [ ]:
"""
Purpose: To check on the status of the Allen split suggestions pipeline
as they run through

- Downloading
- Decimation
- Soma Finding
- Decomposition
- Suggestion Splits

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

INFO - 2021-05-21 14:14:00,013 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-21 14:14:00,015 - settings - Setting database.user to celiib
INFO - 2021-05-21 14:14:00,016 - settings - Setting database.password to newceliipass
INFO - 2021-05-21 14:14:00,020 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-21 14:14:00,021 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-21 14:14:00,032 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-21 14:14:00,444 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-21 14:14:00,448 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-21 14:14:00,449 - settings - Setting database.user to celiib
INFO - 2021-05-21 14:14:00,450 - settings - Setting database.password to newceliipass
INFO - 2021-05-21 14:14:00,451 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-21 14:14:00,451 - settings - Setting database.user to celiib
INFO - 2021-05-21 14:14:00,452 - settings - Setting database.password to newceliipass
INFO - 2021-05-21 14:14:00,453 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-21 14:14:00,899 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-05-21 14:14:01,048 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-21 14:14:01,050 - settings - Setting database.user to celiib
INFO - 2021-05-21 14:14:01,051 - settings - Setting database.password to newceliipass
INFO - 2021-05-21 14:14:01,054 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-21 14:14:01,563 - settings - Setting enable_python_native_blobs to True


# Investigating why some neurons did not get split suggestions

In [5]:
allen_table = (minnie.AllenProofreading() & minnie.AllenProofreadingCurrentDate())
allen_table

segment_id segment id for those to be decimated,month the month that the cell was added,day the day that the cell was added,year the year that the cell was added,n_somas the number of somas associated with the neuron from the Allen Institute
864691134917394186,5,20,2021,2
864691134917552394,5,20,2021,2
864691134939999587,5,20,2021,2
864691134947444476,5,20,2021,2
864691134988431994,5,20,2021,2
864691135012436214,5,20,2021,2
864691135012549622,5,20,2021,2
864691135013122454,5,20,2021,2
864691135013164694,5,20,2021,2
864691135117842397,5,20,2021,2


In [6]:
minnie.BaylorSegmentCentroid() & allen_table  & dict(decimation_ratio=0.25) & "multiplicity=0"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,ver the version number of the materializaiton,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,volume the volume in billions (10*9 nm^3) of the convex hull,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)
864691135122549799,0,0.25,0,88.00,None,None,None,0,0,=BLOB=,0,nan,nan,nan,nan,nan,730.6384756565094
864691135758500174,0,0.25,0,30.00,None,None,None,0,0,=BLOB=,0,nan,nan,nan,nan,nan,1329.2657742500305


In [8]:
key_source = minnie.Decomposition() & (allen_table).proj()
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134917394186,0,0.25,30.00,0,0,1,=BLOB=,347297,691245,0,0,0,0,1,5,139,103,4761159.665713276,2899920.962777224,21695.165,87.059,87.059,310.01,287.018,480,0.00010081577466444628,3.4532374100719423,1888976.265,41,0.0002541058926433891,11.707,68758996686.06743,71526033.29916587,14441.648991783297,36400.1379795354,1677048699.6601813,1256.1993
864691134939999587,0,0.25,30.00,0,0,1,=BLOB=,1064520,2163913,0,1,1,1,2,12,189,83,8620722.716581002,3319052.997881004,27215.146,364.679,299.953,562.58,518.512,4712,0.0005465899037602719,24.93121693121693,6587569.668,120,0.0007152865529284904,39.267,1083737115931.0266,141877100.43232694,125713.02332304213,164512.43334782848,9031142632.758554,6464.8771
864691134947444476,0,0.25,30.00,0,0,1,=BLOB=,1452429,2912400,0,1,0,1,2,17,281,114,12832003.878648698,4786845.5815107245,33082.926,296.07,278.817,594.66,582.725,4817,0.00037538953740616114,17.1423487544484,10679010.581,201,0.00045107175083901156,23.965,545107299628.61456,61804405.338347115,42480.29417569177,51044.73822682268,2711976615.0677342,7373.8979
864691134988431994,0,0.25,30.00,0,0,1,=BLOB=,982013,1994776,0,2,1,2,2,11,202,53,8740123.954824159,2149031.9590088027,29729.876,374.4,290.121,549.805,515.402,4740,0.0005423264045796205,23.465346534653467,6260614.348,129,0.0007571141962312738,36.744,848624243615.4156,109460944.26225038,97095.21832891316,135549.67555005444,6578482508.646633,4329.4674
864691135012436214,0,0.25,30.00,0,0,1,=BLOB=,780868,1591736,0,1,1,1,2,6,408,353,11020043.141487595,8884593.902439833,17643.571,135.08,134.281,397.745,392.066,629,5.707781647714052e-05,1.5416666666666667,4806684.541,194,0.0001308594301612189,3.242,129795629774.06355,56027904.61383692,11778.141710300279,27003.15127130444,669049638.0106369,5933.2451
864691135013122454,0,0.25,88.00,7,0,1,=BLOB=,926526,1879605,0,1,0,1,2,12,162,40,7257760.511799474,1772423.4105831266,31003.817,378

In [9]:
minnie.Decimation()

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691131559524855,0,0.25,24,48,=BLOB=
864691131625403045,0,0.25,11,18,=BLOB=
864691131658969700,0,0.25,14,24,=BLOB=
864691131705161142,0,0.25,11,18,=BLOB=
864691131771477291,0,0.25,4,4,=BLOB=
864691131800703080,0,0.25,4,4,=BLOB=
864691131820044899,0,0.25,5,6,=BLOB=
864691131828706323,0,0.25,7,10,=BLOB=
864691131834277640,0,0.25,12,20,=BLOB=
864691131834278152,0,0.25,4,4,=BLOB=


In [ ]:
key_source & "n_multi_soma_touching_limbs = 0 AND n_same_soma_multi_touching_limbs=0 "

In [ ]:
import neuron_utils as nru

In [ ]:
segment_id = 864691135782479155
dec_mesh = du.fetch_segment_id_mesh(segment_id)
neuron_obj = nru.decompress_neuron("/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135782479155_0_25.pbz2",
                     original_mesh=dec_mesh)

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

In [ ]:
neuron_obj.n_error_limbs

In [ ]:
nviz.visualize_neuron_lite(neuron_obj)

In [ ]:
segment_id_with_no_errors = [
    864691135345291423, #had just soma on the very tip
    864691135373423816, #removed in glia
    864691135753669837,
    ,
]

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'") & "status='error'"
(curr_table).delete()# & "status='error'")

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
seg_ids_to_delete = [k["segment_id"] for k in curr_table.fetch("key")]
seg_ids_to_delete

In [ ]:
minnie.Decomposition()

# All the cells that need to get suggestions

In [ ]:
key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

In [ ]:
no_splits = (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)) - key_source.proj()
no_splits

In [ ]:
minnie.DecompositionSplit() & key_source.proj()

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
(curr_table)#.delete()# & "status='error'")

# Getting the split suggesstions

In [ ]:
import datajoint as dj
dj.U("segment_id") & minnie.DecompositionSplit() & key_source.proj()

In [13]:
import datajoint as dj
dj.U("segment_id") & minnie.DecompositionSplit()
allen_segment_ids = ((dj.U("segment_id") & minnie.DecompositionSplit()) & 
                     (minnie.AllenProofreading() & minnie.AllenProofreadingCurrentDate())).fetch("segment_id")
allen_segment_ids.shape

(289,)

In [14]:
allen_segment_ids = (allen_table).fetch("segment_id")
du.create_suggested_splits_neuroglancer_spreadsheet(segment_ids=allen_segment_ids,output_type="server",
                                                   print_links=True,
                                                   output_filename="allen_spreadsheet_5_20.csv")

  0%|          | 0/289 [00:00<?, ?it/s]

segment_id = 864691134939999587
segment_id = 864691134947444476
segment_id = 864691134988431994
segment_id = 864691135012436214
segment_id = 864691135013122454
segment_id = 864691135013164694
segment_id = 864691135117842397
segment_id = 864691135122297383
segment_id = 864691135122548519
segment_id = 864691135122565159
segment_id = 864691135132778656
segment_id = 864691135134695705
segment_id = 864691135155538532
segment_id = 864691135155813732
segment_id = 864691135162265645
segment_id = 864691135162276141
segment_id = 864691135162487341
segment_id = 864691135210035136
segment_id = 864691135210076864
segment_id = 864691135210079936
segment_id = 864691135210128832
segment_id = 864691135210157760
segment_id = 864691135233286233
segment_id = 864691135256191663
segment_id = 864691135256371375
segment_id = 864691135269815461
segment_id = 864691135272187153
segment_id = 864691135274080229
segment_id = 864691135278416033
segment_id = 864691135291738038
segment_id = 864691135291766966
segment_